# STEP REST client examples using python
---

In [2]:
import os, re, sys, gc, time, json, xmltodict, unittest, logging
from collections import OrderedDict
from datetime import datetime
from Perdy.pretty import prettyPrintLn
from Baubles.Colours import Colours

sys.path.insert(0, '..')
from STEP.REST import *

In [3]:
colours = Colours()

config = {
	'-H':'https://stibo-australia-demo.scloud.stibo.com',
	'-U':'DAVE',
	'-C':'GL',
}

workflow_id = 'WX_Product_WF'
state_id = 'WX_Manual_Approve'
product_id = 'WX_0'
reference_id = 'WX_Product_Tag_Classification'
event_id = 'On_Hold'
cache = dict()

In [5]:
# please use your own favourite password cache here
from Spanners.Squirrel import Squirrel
squirrel = Squirrel()
config['-P'] = squirrel.get(f'stibo:{config["-H"]}:{config["-U"]}')

In [6]:
def render(result):
	if not result: 
		return
	if type(result) in [OrderedDict, list, dict]:
		prettyPrintLn(result)
	else:
		print(result)

---
# Test_01_Classifications

In [7]:
classifications = Classifications(asXML=False)
classifications.hostname = config['-H']
classifications.username = config['-U']
classifications.context = config['-C']
classifications.password = config['-P']

## test_01_create_hierarchy

In [8]:
root = classifications.get('WX_Tags')
#render(root)
print(f'{colours.Green}{root["name"]}{colours.Off}')
assert(root)
#render(root)
assert(root['id'] == 'WX_Tags')

now = datetime.now()
dts = f'{now:%Y-%m-%d %H:%M:%S}'
render(dict(created=dts))

classification = classifications.create(root['id'], 'WX_Tag', f'created {dts}')
render(dict(classification=classification))
assert(classification)

if 'classifications' not in cache.keys():
	cache['classifications'] = list()

cache['classifications'].append(classification['id'])

WX Tags
{
  "created" : "2023-11-15 10:21:34"
}
{
  "classification" : {
    "id" : "WX_Tag_Class_10020676",
    "name" : "created 2023-11-15 10:21:34",
    "objectType" : "WX_Tag",
    "parent" : "WX_Tags",
    "values" : {
      "WX_path" : {
        "calculated" : true,
        "contextLocal" : true,
        "value" : {
          "value" : "Classification 1 root/WX Class Root/WX Tags/created 2023-11-15 10:21:34",
          "valueId" : null,
          "unit" : null
        }
      },
      "WX_object_type" : {
        "calculated" : true,
        "contextLocal" : true,
        "value" : {
          "value" : "WX Tag",
          "valueId" : null,
          "unit" : null
        }
      }
    },
    "references" : {
    },
    "attributeLinks" : [
    ]
  }
}


---
# Test_01_Products

In [9]:
products = Products(asXML=False)
products.hostname = config['-H']
products.username = config['-U']
products.context = config['-C']
products.password = config['-P']

## test_01_find_hierarchy

In [10]:
root = products.get('WX_Root')
print(f'{colours.Green}{root["name"]}{colours.Off}')
assert(root)
#render(root)
assert(root['id'] == 'WX_Root')

children = products.children(root['id'])
render(children)
assert(children)
assert(len(children))

l1 = products.children(children[0])
render(l1)
assert(l1)
assert(len(l1))

l2 = products.children(l1[0])
render(l2)
assert(l2)
assert(len(l2))

l3 = products.children(l2[0])
render(l3)
assert(l3)
assert(len(l3))

product = products.get(l3[0])
#render(product)
assert(product)
assert('id' in product.keys())
assert(product['id'] == product_id)

parent_id = product['parent']
cache['parent_id'] = parent_id
render(dict(parent_id=parent_id))

Groceries
[
  "WX_7FC56270E7A70FA81A5935B72EACBE29",
  "WX_148940665E38701A34152297D3DCDB62",
  "WX_D41D8CD98F00B204E9800998ECF8427E"
]
[
  "WX_FA73C792C8239E904EEB67FC4434CA85"
]
[
  "WX_03D994023612FCE11D588B24854C5050"
]
[
  "WX_0",
  "WX_Product_9915614",
  "WX_Product_9915617",
  "WX_Product_9915619",
  "WX_Product_9915621",
  "WX_Product_9915655",
  "WX_Product_9941520",
  "WX_Product_10016449",
  "WX_Product_9915623",
  "WX_Product_9941522",
  "WX_Product_10016447",
  "WX_Product_9915642",
  "WX_Product_9926916"
]
{
  "parent_id" : "WX_03D994023612FCE11D588B24854C5050"
}


## test_02_create_product

In [11]:
assert('parent_id' in cache.keys())
parent_id = cache['parent_id']

now = datetime.now()
dts = f'{now:%Y-%m-%d %H:%M:%S}'
render(dict(created=dts))

child = products.create(parent_id, 'WX_Product', f'created {dts}', values=[
	f'WX_activation_date={dts}',
	f'WX_brand_name=created by rest',
])

render(child)
assert(child)
assert('id' in child.keys())

if 'products' not in cache.keys():
	cache['products'] = list()

cache['products'].append(child['id'])


{
  "created" : "2023-11-15 10:21:52"
}
{
  "id" : "WX_Product_10020677",
  "name" : "created 2023-11-15 10:21:52",
  "objectType" : "WX_Product",
  "parent" : "WX_03D994023612FCE11D588B24854C5050",
  "values" : {
    "WX_activation_date" : {
      "calculated" : false,
      "contextLocal" : true,
      "value" : {
        "value" : "2023-11-15 10:21:52",
        "valueId" : null,
        "unit" : null
      }
    },
    "WX_path" : {
      "calculated" : true,
      "contextLocal" : true,
      "value" : {
        "value" : "Primary Hierarchy/Products/Groceries/A/B/C/created 2023-11-15 10:21:52",
        "valueId" : null,
        "unit" : null
      }
    },
    "WX_stock_total" : {
      "calculated" : true,
      "contextLocal" : true,
      "value" : {
        "value" : "0",
        "valueId" : null,
        "unit" : null
      }
    },
    "WX_fn_contains_nuts" : {
      "calculated" : true,
      "contextLocal" : true,
      "value" : {
        "value" : "0",
        "valueId" :

## test_03_update_product

In [12]:
assert('products' in cache.keys())

now = datetime.now()
dts = f'{now:%Y-%m-%d %H:%M:%S}'
render(dict(updated=dts))

for id in cache['products']:

	update = products.update(id, 'WX_description', f'modified {dts}')
	render(update)
	assert(update)

{
  "updated" : "2023-11-15 10:21:56"
}
{
  "calculated" : false,
  "contextLocal" : true,
  "value" : {
    "value" : "modified 2023-11-15 10:21:56",
    "valueId" : null,
    "unit" : null
  }
}


## test_04_range_product

In [13]:
assert('products' in cache.keys())

now = datetime.now()
dts = f'{now:%Y-%m-%d %H:%M:%S}'
render(dict(ranged=dts))

for id in cache['products']:

	result = products.references(id, reference_id)
	if 'references' in result.keys():
		for reference in result['references']:
			render(dict(existing=reference))
			if reference['target'] in cache['classifications']:
				removed = products.reference(id, reference_id, reference['target'], targetType='C', remove=True)
				render(removed)

	for classification_id in cache['classifications']:
		reference = products.reference(product_id, reference_id, classification_id, targetType='C')
		render(dict(creating=reference))

{
  "ranged" : "2023-11-15 10:21:59"
}
{
  "creating" : {
    "contextLocal" : true,
    "target" : "WX_Tag_Class_10020676",
    "targetType" : "classification",
    "values" : {
    },
    "entity" : null
  }
}


## test_05_delete_product

In [14]:
assert('products' in cache.keys())

now = datetime.now()
dts = f'{now:%Y-%m-%d %H:%M:%S}'
render(dict(deleted=dts))

items = cache['products']

for i in range(len(items)):
	id = items.pop(0)
	#product_id = items[i]
	print(id)
	update = products.delete(id)
	render(update)
	#assert(update)

{
  "deleted" : "2023-11-15 10:22:02"
}
WX_Product_10020677
{"httpStatus":400,"message":"Found objections. 1/1 : step://product?contextid=GL&id=WX_Product_10020677&workspaceid=Main is in Workflow step://stepworkflow?id=WX_Product_WF. : step://product?contextid=GL&id=WX_Product_10020677&workspaceid=Main is in STEP Workflow step://stepworkflow?id=WX_Product_WF.","httpReason":"Bad Request"}


<Response [400]>: {"httpStatus":400,"message":"Found objections. 1/1 : step://product?contextid=GL&id=WX_Product_10020677&workspaceid=Main is in Workflow step://stepworkflow?id=WX_Product_WF. : step://product?contextid=GL&id=WX_Product_10020677&workspaceid=Main is in STEP Workflow step://stepworkflow?id=WX_Product_WF.","httpReason":"Bad Request"}


---
# Test_03_Workflows

In [15]:
workflows = Workflow(asXML=False)
workflows.hostname = config['-H']
workflows.username = config['-U']
workflows.context = config['-C']
workflows.password = config['-P']

tasks = Task(asXML=False)
tasks.hostname = config['-H']
tasks.username = 'WX_CORE_1'
tasks.context = config['-C']
tasks.password = config['-P']

## take item out of workflow so we can start with it not in a workflow

In [16]:
task_ids = tasks.search(workflow_id, state_id='', node_id=product_id, id_as_base64=True)
for task_id in task_ids:
	task = tasks.get(task_id)
	#render(task)

	if 'instance' in task.keys():
		instance_id = task['instance']
		workflows.terminate(workflow_id, instance_id)
		print(f'{colours.Green}Killing: {instance_id}{colours.Off}')

Killing: eyJ3b3JrZmxvd0lkIjoiV1hfUHJvZHVjdF9XRiIsIm5vZGVJZCI6IldYXzAiLCJub2RlVHlwZSI6InByb2R1Y3QifQ==


## test_01_start_workflow

In [17]:
workflow = workflows.get(workflow_id)
render(workflow)
assert workflow

if 'instances' not in cache.keys():
	cache['instances'] = list()

instance_id = workflows.start(workflow_id, product_id, id_as_base64=True)
print(f'{colours.Green}Starting: {instance_id}{colours.Off}')
assert(instance_id)
cache['instances'].append(instance_id)

print('waiting ...')
time.sleep(3)
print('assumed started')

{
  "id" : "WX_Product_WF",
  "name" : "WX_Product_WF",
  "validObjectTypes" : [
    "WX_Product"
  ]
}
Starting: eyJ3b3JrZmxvd0lkIjoiV1hfUHJvZHVjdF9XRiIsIm5vZGVJZCI6IldYXzAiLCJub2RlVHlwZSI6InByb2R1Y3QifQ==
waiting ...
assumed started


## test_03_search_tasks

In [18]:
cache['tasks'] = list()

task_ids = tasks.search(workflow_id, state_id='', node_id=product_id, id_as_base64=True)
for task_id in task_ids:
	task = tasks.get(task_id)
	render(task)

	if 'instance' in task.keys():
		assert(task['instance'] in cache['instances'])

		print(f'Task State: {colours.Green}{task["state"]}{colours.Off}')
		assert(task['state'] == state_id)

		cache['tasks'].append(task['id'])

{
  "id" : "eyJ3b3JrZmxvd0lkIjoiV1hfUHJvZHVjdF9XRiIsInN0YXRlSWQiOiJXWF9NYW51YWxfQXBwcm92ZSIsIm5vZGVJZCI6IldYXzAiLCJub2RlVHlwZSI6InByb2R1Y3QifQ==",
  "node" : {
    "id" : "WX_0",
    "type" : "product"
  },
  "workflow" : "WX_Product_WF",
  "state" : "WX_Manual_Approve",
  "userAssignee" : null,
  "groupAssignee" : "WX_Product_Core",
  "deadline" : null,
  "statusFlag" : null,
  "taskCreationTime" : "2023-11-15T10:23:26+11:00",
  "instance" : "eyJ3b3JrZmxvd0lkIjoiV1hfUHJvZHVjdF9XRiIsIm5vZGVJZCI6IldYXzAiLCJub2RlVHlwZSI6InByb2R1Y3QifQ=="
}
Task State: WX_Manual_Approve


## test_04_interact_tasks

In [19]:
task_ids = cache['tasks']
for i in range(len(task_ids)):
	#task_id = task_ids[i]
	task_id = task_ids.pop(0)

	claimed = tasks.claim(task_id)
	render(dict(claimed=claimed))

	events = tasks.events(task_id)
	render(dict(events=events))
	event_ids = list(map(lambda x:x['id'], events))
	assert(event_id in event_ids)

	now = datetime.now()
	dts = f'{now:%Y-%m-%d %H:%M:%S}'
	render(dict(triggering=dts))

	if True: # either trigger of release here
		triggered = tasks.trigger(task_id, event_id, message=f'triggered at {dts}')
		render(dict(triggered=triggered))

		print('waiting ...')
		time.sleep(3)

		task_ids = tasks.search(workflow_id, state_id='', node_id=product_id, id_as_base64=True)
		for task_id in task_ids:
			task = tasks.get(task_id)
			render(dict(search=task))
			print(f'Task State: {colours.Green}{task["state"]}{colours.Off}')
			assert(task['state'] == 'WX_OnHold')

	else:
		released = tasks.release(task_id)
		render(dict(released=released))

{
  "claimed" : ""
}
{
  "events" : [
    {
      "id" : "On_Hold",
      "anonymous" : false
    },
    {
      "id" : "Rejected",
      "anonymous" : false
    },
    {
      "id" : "Approved",
      "anonymous" : false
    }
  ]
}
{
  "triggering" : "2023-11-15 10:23:39"
}
{
  "triggered" : {
    "id" : "eyJ3b3JrZmxvd0lkIjoiV1hfUHJvZHVjdF9XRiIsIm5vZGVJZCI6IldYXzAiLCJub2RlVHlwZSI6InByb2R1Y3QifQ=="
  }
}
waiting ...
{
  "search" : {
    "id" : "eyJ3b3JrZmxvd0lkIjoiV1hfUHJvZHVjdF9XRiIsInN0YXRlSWQiOiJXWF9PbkhvbGQiLCJub2RlSWQiOiJXWF8wIiwibm9kZVR5cGUiOiJwcm9kdWN0In0=",
    "node" : {
      "id" : "WX_0",
      "type" : "product"
    },
    "workflow" : "WX_Product_WF",
    "state" : "WX_OnHold",
    "userAssignee" : "WX_CORE_1",
    "groupAssignee" : null,
    "deadline" : null,
    "statusFlag" : null,
    "taskCreationTime" : "2023-11-15T10:23:42+11:00",
    "instance" : "eyJ3b3JrZmxvd0lkIjoiV1hfUHJvZHVjdF9XRiIsIm5vZGVJZCI6IldYXzAiLCJub2RlVHlwZSI6InByb2R1Y3QifQ=="
  }
}
Task State: WX_O

---
# Test_04_Endpoints

In [20]:
endpoints = Endpoints(asXML=True)
endpoints.hostname = config['-H']
endpoints.username = config['-U']
endpoints.password = config['-P']
#endpoints.verbose = True

# test_01_endpoints

In [21]:
print(f'{colours.Green}{endpoints}{colours.Off}')
assert endpoints

result = endpoints.list()
assert(result)

_result = xmltodict.parse(result)
assert('IntegrationEndpoints' in _result.keys())
assert(len(_result['IntegrationEndpoints']))

#render(_result)